In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import sys
path = os.path.abspath('..')
sys.path.append(path)
import re

from essay_grader.punctuation import clean_string
from essay_grader.bibliography import remove_all
from nltk.corpus import stopwords
from nltk import download
download('stopwords')
stop_words = stopwords.words('english')

from gensim.models import Word2Vec
from wmd import WMD
import gensim.downloader as api

import spacy
# from essay_grader.remove_stopwords import remove_stopwords
# from essay_grader.vectorize import doc2vec_vectorize

# from essay_grader.lemmatize import lemmatize
# from essay_grader.docxtocsv import dtoc


# from nltk.corpus import wordnet 
# import stanza
# stanza.download('en')

# import pickle
import re
# import matplotlib.pyplot as plt

# from itertools import chain, product, combinations
# import itertools
from time import time


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davidchadsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
nlp = spacy.load('en_core_web_lg')

In [27]:
WMD()

spacy.tokens.doc.Doc

## Preprocessing text

In [2]:
#import text, clean text and essay titles, remove bibliographies from text
df = pd.read_csv('../essay_grader/data/essay_data.csv')
df = clean_string(df, 'text')
df = clean_string(df, 'title_name')
df = remove_all(df)


In [84]:
df['text'] = df['text'].apply(lambda  x: x.strip().split('\n'))

In [85]:
df['text'] = df['text'].apply(lambda  x: [i.split('. ') for i in x])

In [86]:
df['number_of_sentences'] =  df['text'].apply(lambda x: len(x))

In [87]:
df['senteces_per_paragraph'] = df['text'].apply(lambda x: [len(i) for i in x])

In [3]:
model = Word2Vec()

DeprecationWarning: Deprecated. Use gensim.models.KeyedVectors.load_word2vec_format instead.

In [134]:
word_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [168]:
sentence_a = df['title_name'][0]
sentence_b = df['text'][0][-1][-1]

sentence_a = [w for w in sentence_a if w not in stop_words]
sentence_b = [w for w in sentence_b if w not in stop_words]
distance = word_vectors.wmdistance(sentence_a, sentence_b)
distance

2.7347112287049558

In [39]:
df.groupby('level')['mean_word_length'].mean(), df.groupby('level')['mean_word_length'].std()

(level
 1    7.002955
 2    7.100815
 3    7.171796
 4    7.262513
 5    7.085403
 Name: mean_word_length, dtype: float64,
 level
 1    0.336261
 2    0.324077
 3    0.300105
 4    0.307276
 5    0.279619
 Name: mean_word_length, dtype: float64)

In [38]:
df.groupby('level')['var_word_length'].mean(), df.groupby('level')['var_word_length'].std()

(level
 1    6.100955
 2    6.322417
 3    6.347202
 4    6.560955
 5    6.235623
 Name: var_word_length, dtype: float64,
 level
 1    0.772998
 2    0.891007
 3    0.894498
 4    0.974064
 5    0.379212
 Name: var_word_length, dtype: float64)

## Create synonyms of titles

In [30]:
new_df = pd.read_pickle('../essay_grader/pickle_data/text_to_text_similarity.pkl')

TypeError: int() argument must be a string, a bytes-like object or a number, not '_NoValueType'

TypeError: int() argument must be a string, a bytes-like object or a number, not '_NoValueType'

In [464]:
#create title_df with one row for each unique title name.   
title_df = pd.DataFrame(df['title_name'].unique(), columns = ['title_name'])
#Create new column title_name_short with stop words removed from title and 

title_df['title_name_short'] = title_df['title_name'].apply(remove_stopwords)

#add new column with lemnatized titles and doc_list from stanza
title_df, doc_list = lemmatize(title_df, 'title_name_short', return_doclist=True, processors='tokenize,mwt')
title_df.drop(columns = ['title_name_short'], inplace = True )

2020-08-24 16:17:08 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-08-24 16:17:08 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |

2020-08-24 16:17:08 INFO: Use device: cpu
2020-08-24 16:17:08 INFO: Loading: tokenize
2020-08-24 16:17:08 INFO: Done loading processors!


In [427]:
#create a new column, title_key_word, with only the nouns and verbs from the title.
title_df['title_key_word'] = None
for i, _  in title_df.iterrows():
    for sentence in doc_list[i].sentences:
        key_words = []
        for word in sentence.words:
            if word.pos == 'NOUN' or word.pos == 'VERB':
                key_words.append((word.lemma, word.pos[0].lower()))
        title_df.at[i, 'title_key_word'] = key_words


In [429]:
#create a column in title_df with the synonyms for all key words of the title that follow the same pos called title_synonyms
title_df['title_synonyms'] = None
for i,_ in title_df.iterrows():
    synonyms = []
    for j in range(len(title_df['title_key_word'][i])):
        word, pos = title_df.at[i, 'title_key_word'][j]
        syns = wordnet.synsets(word)
        word_synonyms = []
        for syn in syns:
            word_synonyms += [lm.name() for lm in syn.lemmas()]
        word_synonyms  = list(set(word_synonyms))
        synonyms += word_synonyms
    #remove repeates and items that are two or more characters
    synonyms  = list(set(synonyms))
    synonyms = [ x for x in synonyms if "_" not in x ]
    #remove words that are common amongst all essays
    try:
        common_words = ['knowledge', 'area', 'way', 'know', 'knowing']
        synonyms = [ x for x in synonyms if x not in common_words]
    except:
        pass
    title_df.at[i,'title_synonyms'] = synonyms

In [ ]:
df = df.merge(title_df)

## Calculate how often each title word and synonym appear in the text

In [41]:
#calculate how often each key word and synonym of key words appear in each essay.  Store vaues in new column frequency_title_words
df['frequency_title_words'] = 0
for i,_ in df.iterrows():
    count = 0
    for j in title_df['title_synonyms'][df['title_no'][i]]:
        count += df.at[i, 'text'].count(j)
    df.at[i,'frequency_title_words'] = count

In [42]:
df.groupby('level')['frequency_title_words'].mean(), df.groupby('level')['frequency_title_words'].std()

(level
 1     74.551020
 2     81.040909
 3     82.886792
 4     95.406250
 5    122.200000
 Name: frequency_title_words, dtype: float64,
 level
 1    43.553636
 2    48.508467
 3    50.712641
 4    56.351989
 5    77.667239
 Name: frequency_title_words, dtype: float64)

## Explore gensim pre-trained options

In [4]:
import gensim.downloader as api

In [5]:
corpus = api.load('text8')

[==================================================] 100.0% 31.6/31.6MB downloaded


In [6]:
model = Word2Vec(corpus)

In [7]:
import json
info = api.info()
print(json.dumps(info, indent=4))

{
    "corpora": {
        "semeval-2016-2017-task3-subtaskBC": {
            "num_records": -1,
            "record_format": "dict",
            "file_size": 6344358,
            "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py",
            "license": "All files released for the task are free for general research use",
            "fields": {
                "2016-train": [
                    "..."
                ],
                "2016-dev": [
                    "..."
                ],
                "2017-test": [
                    "..."
                ],
                "2016-test": [
                    "..."
                ]
            },
            "description": "SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collect

In [8]:
for corpus_name, corpus_data in sorted(info['corpora'].items()):
    print(
        '%s (%d records): %s' % (
            corpus_name,
            corpus_data.get('num_records', -1),
            corpus_data['description'][:40] + '...',
        )
    )

20-newsgroups (18846 records): The notorious collection of approximatel...
__testing_matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Synopsis of t...
__testing_multipart-matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Synopsis of t...
fake-news (12999 records): News dataset, contains text and metadata...
patent-2017 (353197 records): Patent Grant Full Text. Contains the ful...
quora-duplicate-questions (404290 records): Over 400,000 lines of potential question...
semeval-2016-2017-task3-subtaskA-unannotated (189941 records): SemEval 2016 / 2017 Task 3 Subtask A una...
semeval-2016-2017-task3-subtaskBC (-1 records): SemEval 2016 / 2017 Task 3 Subtask B and...
text8 (1701 records): First 100,000,000 bytes of plain text fr...
wiki-english-20171001 (4924894 records): Extracted Wikipedia dump from October 20...


In [11]:
for model_name, model_data in sorted(info['models'].items()):
    print(
        '%s (%d records): %s' % (
            model_name,
            model_data.get('num_records', -1),
            model_data['description'][:40] + '...',
        )
    )

__testing_word2vec-matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Word vecrors ...
conceptnet-numberbatch-17-06-300 (1917247 records): ConceptNet Numberbatch consists of state...
fasttext-wiki-news-subwords-300 (999999 records): 1 million word vectors trained on Wikipe...
glove-twitter-100 (1193514 records): Pre-trained vectors based on  2B tweets,...
glove-twitter-200 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-25 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-50 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-wiki-gigaword-100 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-200 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-300 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-50 (400000 records): Pre-trained vectors based on Wikipedia 2...
word2vec-google-news-300 (3000000 records): Pre-trai

In [18]:
fake_news_info = api.info('text8')
print(json.dumps(fake_news_info, indent=4))

{
    "num_records": 1701,
    "record_format": "list of str (tokens)",
    "file_size": 33182058,
    "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py",
    "license": "not found",
    "description": "First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.",
    "checksum": "68799af40b6bda07dfa47a32612e5364",
    "file_name": "text8.gz",
    "read_more": [
        "http://mattmahoney.net/dc/textdata.html"
    ],
    "parts": 1
}


In [ ]:
fake_news_info = api.info('fake-news')
print(json.dumps(fake_news_info, indent=4))